# List price:

In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()
header = {'User-Agent':str(ua.chrome)} #requests to fake a browser visit
url = "https://www.tigerdirect.com/applications/SearchTools/item-details.asp?EdpNo=1501390"
keyword = "p.list-price span.sr-only"

def get_content_from_website(url, keyword):
	try:
		container = []
		page = requests.get(url, headers=header)
        # Create a beautifulsoup object 
		soup = BeautifulSoup(page.text, 'lxml')
		# find class p that has "list-price" and class span that has "sr-only".
		list_of_contents = soup.select(keyword)
		for i in list_of_contents:
			container.append(i.text)
	
	except:
		print("Problem with the connection...")
	return container

container = get_content_from_website(url, keyword)
print(container)

['$1,399\r\n          and 99 cents\r\n        ']


## regex

In [2]:
import re
first_part = re.findall(r'(?:\d+\.)?\d+,\d+', container[0])
first_part[0

'1,399'

In [3]:
second_part = re.findall(r'(?:\b\d{2})\b', container[0])
second_part[0]

'99'

In [4]:
first_part[0] = first_part[0].replace(",","")
current_price = first_part[0] + "." + second_part[0]

print(current_price)

1399.99


# Current price:

In [5]:
url="https://www.tigerdirect.com/applications/SearchTools/item-details.asp?EdpNo=1501390"
keyword="p.final-price span.sr-only"

container_list = get_content_from_website(url,keyword)
print(container_list)

['$1,029\r\n          and 99 cents\r\n        ']


## regex

In [6]:
import re
first_part_list = re.findall(r'(?:\d+\.)?\d+,\d+', container_list[0])
first_part_list[0]

'1,029'

In [7]:
second_part_list = re.findall(r'(?:\b\d{2})\b', container_list[0])
second_part_list[0]

'99'

In [8]:
first_part_list[0] = first_part_list[0].replace(",","")
list_price = first_part_list[0] + "." + second_part_list[0]

print(list_price)

1029.99


# US News

## Top stories

In [9]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()
url="https://www.usnews.com/"
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}

try:
    container = []
    page = requests.get(url, headers=header, timeout=5)
    # Create a beautifulsoup object 
    soup = BeautifulSoup(page.text, 'lxml')
    # Find the paragraph that have "top stories" in it
    list_of_contents = soup.find_all("p", string="Top Stories")
    for tag in list_of_contents:
        nextone = tag.next_sibling # find the part next to "top stories," which is the division that includes the 2 sub top stories 
        top_stroies = nextone.select("h3.story-headline") # then find the 2 sub top stories that are in class h3 with "story-headline"
    for i in top_stroies:
        container.append(i.text)
except:
    print("Problem with the connection...")
print(container)



['U.S. Hits Debt Limit as Parties Squabble', 'Biden: ‘No Regrets’ Over Documents']


## Second top story's URL

In [10]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()
url="https://www.usnews.com/"
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}

second_story_title = ''.join(container[1])

try:
    links = []
    page = requests.get(url, headers=header, timeout=5)
    # Create a beautifulsoup object 
    soup = BeautifulSoup(page.text, 'lxml')
    list_of_contents = soup.find_all("h3", string=second_story_title) # find the second top story
    for tag in list_of_contents: # since list_of_contents is a list, need to get a class to use the function find.all
        for i in tag.find_all('a'):
            links.append(i.get('href')) # get the url link for the second top story
except:
    print("Problem with the connection...")

links = ''.join(links)
print(type(links))
print(links)


<class 'str'>
https://www.usnews.com/news/politics/articles/2023-01-19/biden-says-he-has-no-regrets-about-his-handling-of-documents


## Second story's header + 3 sentences

In [11]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests

ua = UserAgent()
url= links
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}

try:
    title = []
    body = []
    page = requests.get(url, headers=header, timeout=5)
    # Create a beautifulsoup object 
    soup = BeautifulSoup(page.text, 'lxml')
    list_of_contents = soup.select("h1.Heading-sc-1w5xk2o-0") # Find the header with h1 class
    for i in list_of_contents:
        title.append(i.text) # store the header to "title"
        
    body_contents = soup.find_all("div", class_="Raw-slyvem-0 bCYKCn") # Find the article's contents
    for tag in body_contents: # get all paragraphs
        for i in tag.find_all('p'):
            body.append(i.text)
            
    holder = []
    final_result = []
    for a in body: # use ". " to split all sentences
        split_body = a.split(". ")
        holder+=split_body # store separate sentences to "holder"
    counter = 0
    for sentence in holder:
        if sentence != "":
            if sentence.strip()[-1] != ".": # add back period to sentences
                final_result.append(sentence+".")
            else:
                final_result.append(sentence)
            counter += 1
        if counter >= 3: # only select the first 3 sentences
            break
    
except:
    print("Problem with the connection...")
    

title = ''.join(title)
print(title)

final_result = ''.join(final_result)
print(final_result)


Biden: ‘No Regrets’ Over Documents
President Biden told reporters Thursday that he has no regrets about how the administration handled the disclosure of the discovery of classified documents at his home and off-White House campus office, saying he was confident the investigations would show nothing untoward had occurred."As soon as we found the handful of documents were filed in the wrong place, we immediately turned them over to the Archives and the Justice Department," Biden said from a pier at Seacliff State Beach in Aptos, California, where he was delivering remarks after touring storm damage in the state.Chiding reporters briefly for asking about the documents instead of the emergency response to the California storm, Biden added, "We're fully cooperating and looking forward to this being resolved quickly.
